# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt",encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 116, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 122, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 126, which is longer than the specified 100
Created a chunk of size 128, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 114, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 123, which is longer tha

In [5]:
docs[0]

Document(page_content='[客户问题] 这款鞋子为什么这么贵？\n[销售回答] Nike鞋子的价格反映了我们在研发和使用高品质材料上的投入。这款鞋子不仅采用了创新技术来提升运动表现，还有着时尚的设计。实际上，它提供的长期价值和耐用性使得初期投资非常划算。')

In [6]:
len(docs)

70

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url="https://api.xiaoai.plus/v1"))

D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
query = "我想买跑步鞋"

In [10]:
answer_list = db.similarity_search(query)

In [11]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我需要一双适合跑步的鞋子。
[销售回答] 您来对地方了！Nike有多款专为跑步设计的鞋子，它们有着优异的缓震性能和支撑性。根据您的跑步习惯和路面类型，我可以为您推荐几款最受欢迎的跑鞋。

[客户问题] 我刚开始跑步，不知道应该选择什么样的鞋子。
[销售回答] 对于刚开始跑步的朋友，选择一双既有良好支撑性又不失舒适度的鞋子非常重要。我们的Nike Winflo和Nike Revolution系列是专为新手跑者设计的，它们提供了必要的缓震和稳定性，非常适合初学者。

[客户问题] 我想要一双有助于提升跑步速度的鞋子。
[销售回答] 提升跑步速度需要轻便且具有良好能量回馈的鞋子。我们的Nike ZoomX Vaporfly Next%是许多世界顶级跑者的选择，它们利用先进的ZoomX泡沫和碳纤维板提供无与伦比的反应性和速度。

[客户问题] 我想要一双适合快速短跑的鞋子。
[销售回答] 短跑需要的是一双轻便、快速的鞋子。Nike Zoom系列，如Nike Zoom Rival和Nike Zoom Fly，专为速度而设计，能够帮助您在短跑中发挥最佳表现。



In [12]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [13]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [14]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001927FDB9D50>, search_kwargs={'k': 3})

In [15]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 我需要一双适合跑步的鞋子。
[销售回答] 您来对地方了！Nike有多款专为跑步设计的鞋子，它们有着优异的缓震性能和支撑性。根据您的跑步习惯和路面类型，我可以为您推荐几款最受欢迎的跑鞋。

[客户问题] 我刚开始跑步，不知道应该选择什么样的鞋子。
[销售回答] 对于刚开始跑步的朋友，选择一双既有良好支撑性又不失舒适度的鞋子非常重要。我们的Nike Winflo和Nike Revolution系列是专为新手跑者设计的，它们提供了必要的缓震和稳定性，非常适合初学者。

[客户问题] 我想要一双有助于提升跑步速度的鞋子。
[销售回答] 提升跑步速度需要轻便且具有良好能量回馈的鞋子。我们的Nike ZoomX Vaporfly Next%是许多世界顶级跑者的选择，它们利用先进的ZoomX泡沫和碳纤维板提供无与伦比的反应性和速度。



In [16]:
docs = topK_retriever.get_relevant_documents("你们有没有300块钱左右的跑鞋啊？")

In [17]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我需要一双适合跑步的鞋子。
[销售回答] 您来对地方了！Nike有多款专为跑步设计的鞋子，它们有着优异的缓震性能和支撑性。根据您的跑步习惯和路面类型，我可以为您推荐几款最受欢迎的跑鞋。

[客户问题] 你们有没有适合孩子的鞋子？
[销售回答] 当然有，我们有一系列专为儿童设计的运动鞋，既有趣又实用。它们有着优秀的耐用性和舒适性，能够支持孩子们活泼的生活方式。

[客户问题] 我经常在沙滩上跑步，需要一双什么样的鞋子？
[销售回答] 沙滩跑步时鞋子需要有良好的抓地力和稳定性。我们的Nike Free系列鞋子设计轻盈，底部的凹槽能够适应不同的地形变化，非常适合沙滩跑步。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [18]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [19]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我需要一双适合跑步的鞋子。
[销售回答] 您来对地方了！Nike有多款专为跑步设计的鞋子，它们有着优异的缓震性能和支撑性。根据您的跑步习惯和路面类型，我可以为您推荐几款最受欢迎的跑鞋。

[客户问题] 我刚开始跑步，不知道应该选择什么样的鞋子。
[销售回答] 对于刚开始跑步的朋友，选择一双既有良好支撑性又不失舒适度的鞋子非常重要。我们的Nike Winflo和Nike Revolution系列是专为新手跑者设计的，它们提供了必要的缓震和稳定性，非常适合初学者。

[客户问题] 我想要一双有助于提升跑步速度的鞋子。
[销售回答] 提升跑步速度需要轻便且具有良好能量回馈的鞋子。我们的Nike ZoomX Vaporfly Next%是许多世界顶级跑者的选择，它们利用先进的ZoomX泡沫和碳纤维板提供无与伦比的反应性和速度。

[客户问题] 我想要一双适合快速短跑的鞋子。
[销售回答] 短跑需要的是一双轻便、快速的鞋子。Nike Zoom系列，如Nike Zoom Rival和Nike Zoom Fly，专为速度而设计，能够帮助您在短跑中发挥最佳表现。



### 提取向量数据库中的`销售回答`

In [20]:
docs = retriever.get_relevant_documents(query)

In [21]:
docs[0].page_content

'[客户问题] 我需要一双适合跑步的鞋子。\n[销售回答] 您来对地方了！Nike有多款专为跑步设计的鞋子，它们有着优异的缓震性能和支撑性。根据您的跑步习惯和路面类型，我可以为您推荐几款最受欢迎的跑鞋。'

In [22]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我需要一双适合跑步的鞋子。\n',
 '您来对地方了！Nike有多款专为跑步设计的鞋子，它们有着优异的缓震性能和支撑性。根据您的跑步习惯和路面类型，我可以为您推荐几款最受欢迎的跑鞋。']

In [23]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [24]:
ans

'您来对地方了！Nike有多款专为跑步设计的鞋子，它们有着优异的缓震性能和支撑性。根据您的跑步习惯和路面类型，我可以为您推荐几款最受欢迎的跑鞋。'

#### 尝试各种问题

In [25]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [26]:
query = "我想鞋底软一点的"

print(sales(query))

[]


D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [27]:
print(sales(query, 0.75))

['安全是我们设计鞋子时考虑的重点之一。我们的鞋款都经过精心设计，不仅在干燥地面上表现出色，在湿滑的地面上也能提供必要的抓地力。', '在沙地上训练需要鞋子提供额外的抓地力和稳定性。Nike ACG系列中的一些款式具有加强的外底，可以提供在沙地上良好的性能。', '我们理解每个人的脚型都是独特的，Nike有提供针对平足问题的鞋款，比如Nike Structure系列，它们提供了额外的支撑和稳定性，帮助减轻平足引起的不适。', '慢跑时对鞋子的舒适度和缓震性有较高要求。我们的Nike Air Zoom Vomero系列能够提供超柔软的缓震和支撑，非常适合慢跑和日常训练。']


In [28]:
query = "价格300以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们的目标是确保您对购买的Nike产品完全满意。如果您对购买的鞋子不满意，只要在购买后30天内保持鞋子的原始状态，您就可以无条件退换。', '户外篮球场需要鞋子提供额外的耐磨性和抓地力。我们的Nike Air Raid是专为户外篮球设计的，它们的外底设计提供了出色的耐磨性和抓地力。', '当然，我们提供礼品包装服务，可以帮您将鞋子包装成精美的礼物。此外，我们还提供礼品卡服务，让收礼人可以根据自己的喜好来挑选鞋子。', '对于室内自行车训练，我们推荐Nike SuperRep Cycle，它们具备良好的通风性能和固定脚踝的设计，确保您在骑行过程中的稳定性和舒适性。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5,base_url="https://api.xiaoai.plus/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "你们有运动员专用的鞋子吗"})

{'query': '你们有运动员专用的鞋子吗',
 'result': '是的，Nike提供多款专业运动员使用的高性能鞋子。我可以根据您的具体运动类型和需求，推荐适合您的产品。'}

In [32]:
qa_chain({"query": "鞋子质量怎么样"})

{'query': '鞋子质量怎么样',
 'result': '销售回答表明他们非常重视产品的耐用性，并且这款新鞋采用了加强的材料和改进的设计，以提高耐磨性。此外，还提供保养指南帮助延长鞋子的使用寿命。根据这些信息，可以推断鞋子的质量应该是不错的，特别是在耐磨性方面。'}

In [33]:
print(sales("鞋子质量怎么样"))

['我们非常重视产品的耐用性。这款新鞋采用了加强的材料和改进的设计，以提高耐磨性。此外，我们还提供保养指南，帮助您延长鞋子的使用寿命。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(base_url="https://api.xiaoai.plus/v1"),allow_dangerous_deserialization=True)


In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5,base_url="https://api.xiaoai.plus/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [40]:
qa_chain({"query": "我想买篮球鞋，你们有么"})

D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买篮球鞋，你们有么',
 'result': '抱歉，我是一个AI助手，无法为您提供购买篮球鞋的服务。但是，您可以在网上商店或附近的体育用品商店购买篮球鞋。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [41]:
qa_chain({"query": "我想买篮球鞋，你们有么"})

D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买篮球鞋，你们有么',
 'result': '抱歉，我是一个AI语言模型，不能直接为您提供购物服务。但是，您可以在各大电商平台或者体育用品商店购买篮球鞋。例如：京东、淘宝、天猫、亚马逊等。'}

In [51]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [42]:
result = qa_chain({"query": "我想买篮球鞋，你们有么"})

D:\devTool\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [53]:
result

{'query': '我想买别墅，你们有么', 'result': '我不知道。', 'source_documents': []}